In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import gym
from main.alpaca import *
from main.datagen import *
from main.dataViz import *

In [ ]:
cfg_filename = 'configs/multistep-config.yml'
with open(cfg_filename,'r') as ymlfile:
    config = yaml.load(ymlfile)

In [ ]:
DG = DataGenerator(config,None,'Multistep')
N_train = 5000
N_test = 500

noise1 = 0.1
noise2 = 0.3
noise3 = 0.5

Y1,X1,step_mat = DG.sample_trajectories(None,30,N_train,return_lists=True,sigma_eps=noise1)

lists = step_mat

Y2,X2 = DG.sample_trajectories(None,30,N_train,return_lists=False,sigma_eps=noise2,lists=lists)
Y3,X3 = DG.sample_trajectories(None,30,N_train,return_lists=False,sigma_eps=noise3,lists=lists)

Y_test1,X_test1,step_mat_test= DG.sample_trajectories(None,30,N_test,return_lists=True,sigma_eps=noise1)
lists_test = step_mat_test
np.random.seed(1)
Y_test1,X_test1 = DG.sample_trajectories(None,30,N_test,return_lists=False,sigma_eps=noise1,lists=lists_test)
np.random.seed(1)
Y_test2,X_test2 = DG.sample_trajectories(None,30,N_test,return_lists=False,sigma_eps=noise2,lists=lists_test)
np.random.seed(1)
Y_test3,X_test3 = DG.sample_trajectories(None,30,N_test,return_lists=False,sigma_eps=noise3,lists=lists_test)

## Default agent

In [ ]:
g1 = tf.Graph()
sess1 = tf.InteractiveSession(config=tf.ConfigProto(log_device_placement=True), graph=g1)
agent1 = ALPaCA(config)
agent1.sigma_scalar = noise1
agent1.construct_model(sess1,g1)
agent1.train(sess1,Y1,X1,3000)

In [ ]:
g2 = tf.Graph()
sess2 = tf.Session(config=tf.ConfigProto(log_device_placement=True), graph=g2)
agent2 = ALPaCA(config)
agent2.sigma_scalar = noise2
agent2.construct_model(sess2,g2)
agent2.train(sess2,Y2,X2,3000)

In [ ]:
g3 = tf.Graph()
sess3 = tf.Session(config=tf.ConfigProto(log_device_placement=True), graph=g3)
agent3 = ALPaCA(config)
agent3.sigma_scalar = noise3
agent3.construct_model(sess3,g3)
agent3.train(sess3,Y3,X3,3000)

# Visualize all agents

In [ ]:
ind = 2
sample_size_list = [0,2,4,6,8,10,12,14,16,18,20]
plt.figure(figsize=(9,len(sample_size_list)*1))
for i,num_pts in enumerate(sample_size_list):
    X_update1 = X_test1[ind:(ind+1),:num_pts,:]
    Y_update1 = Y_test1[ind:(ind+1),:num_pts,:]
    
    X_update2 = X_test2[ind:(ind+1),:num_pts,:]
    Y_update2 = Y_test2[ind:(ind+1),:num_pts,:]
    
    X_update3 = X_test3[ind:(ind+1),:num_pts,:]
    Y_update3 = Y_test3[ind:(ind+1),:num_pts,:]
    
    title=None
    legend=False
    if i == 0:
        legend=True
        title=True

    ax1 = plt.subplot(len(sample_size_list),3,3*i+1)
    gen_multistep_fig(agent1, sess1, X_update1, Y_update1, step_mat_test[ind,:], label=None)
    if i == 0:
        plt.title(r'ALPaCA, ' + r'$\Sigma_\epsilon = 0.1$')
    if i < len(sample_size_list) - 1:
        plt.setp(ax1.get_xticklabels(), visible=False)
        
    ax2 = plt.subplot(len(sample_size_list),3,3*i+2, sharey=ax1)
    gen_multistep_fig(agent2, sess2, X_update2, Y_update2, step_mat_test[ind,:], label=None)
    plt.setp(ax2.get_yticklabels(), visible=False)
    if i == 0:
        plt.title(r'ALPaCA, ' + r'$\Sigma_\epsilon = 0.3$')
    if i < len(sample_size_list) - 1:
        plt.setp(ax2.get_xticklabels(), visible=False)
    
    
    ax3 = plt.subplot(len(sample_size_list),3,3*i+3, sharey=ax1)
    gen_multistep_fig(agent3, sess3, X_update3, Y_update3, step_mat_test[ind,:], label=None)
    plt.setp(ax3.get_yticklabels(), visible=False)
    if i == 0:
        plt.title(r'ALPaCA, ' + r'$\Sigma_\epsilon = 0.5$')
    if i < len(sample_size_list) - 1:
        plt.setp(ax3.get_xticklabels(), visible=False)

plt.tight_layout(w_pad=0.0,h_pad=0.2)
plt.savefig('figures/multistep_varying_noise.pdf')
plt.show()

In [ ]:
K,L = sess1.run([agent.K, agent.L])
x_range = np.reshape( np.linspace(-5,5,100), (1,100,1) )
phi = sess1.run( agent.phi, {agent.x: x_range} ) #agent.encode(sess,x_range)
plot_sample_fns(x_range,phi,K,L,sess1.run(agent.SigEps),10)

In [ ]:
x_range = np.reshape( np.linspace(-5,5,100), (1,100,1) )
gen_sin_bases_fig(agent, sess1, x_range, 32)